### **1. Importation des librairies et préparations**

In [4]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

path = '../../../../images/image_train/'

list_img = os.listdir(path)

print('le nombre total d\'image dans le repertoire image_train est de :', len(list_img))

#with os.scandir(path) as dir_contents:
    #for i in dir_contents:
        #print(i.name) # nom du fichier ou répertoire
        #print(i.is_file()) # True si entrée est un fichier sinon False
        #print(i.is_dir()) # True si entrée es un répertoire sinon False
        #print(i.stat()) # Retourne un objet os.stat_result qui contient 
        # diverses informations sur l'entrée, telles que la taille, 
        # les informations sur les permissions, la date de modification, etc.

# Chargement des fichiers '="X_train_uptade.csv" et "Y_trainCVw08PX.csv"
X_train = pd.read_csv('../../../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../../../y_train.csv', index_col=0)

# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index = True, right_index = True)

# Affichage des 5 premières lignes
display(df.head())

le nombre total d'image dans le repertoire image_train est de : 84916


,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


In [5]:
# création d'une nouvelle colonne pour manipuler les images
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]


### **2. Création d'une colonne poids des images en bits **

In [6]:
# listes vide pour récuperer les tailles et noms de chaque image du dossier
size_img = []
name_img = []

with os.scandir(path) as dossier_img:
    for i in dossier_img:
        size_img.append(i.stat().st_size) # récupère la taille de l'image
        name_img.append(i.name) # récupère le nom de l'aimge
        

# création des DF pour chaque liste
name_img = pd.DataFrame(name_img, columns = ['Nom image'])
size_img = pd.DataFrame(size_img, columns = ['Taille image (bits)'])
## Rappel : 1 octet = 8 bits & 1 Mo = 2^20 octets = 1 048 576 octets

df_img = pd.concat([name_img, size_img], axis = 1) # concatenation des df en colonne

In [8]:
df_with_img = df.merge(df_img, how = 'inner')
#df.drop(['Nom image'], axis = 1, inplace = True)
#df_with_img.info() 
df_with_img['lien'] = str(path) + '/' + df_with_img['Nom image']
#df_with_img.head()

,designation,description,productid,imageid,prdtypecode,Nom image,Taille image (bits),lien
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10,image_1263597046_product_3804725264.jpg,14010,../../../../images/image_train//image_12635970...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280,image_1008141237_product_436067568.jpg,14854,../../../../images/image_train//image_10081412...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,image_938777978_product_201115110.jpg,6898,../../../../images/image_train//image_93877797...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280,image_457047496_product_50418756.jpg,14404,../../../../images/image_train//image_45704749...
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705,image_1077757786_product_278535884.jpg,20435,../../../../images/image_train//image_10777577...


#### TEST ANOVA

L'objectif de ce test est de conclure sur l'influence d'une variable explicative catégorique sur la distribution d'une variable continue à expliquer.
Le raisonnement que nous effectuons avec l'ANOVA est que si la variable qualitative (prdtypecode) n'affecte pas la variable quantitative (Taille_image_bits), la moyenne devrait être la même pour les 27 modalités.

Soit H0 : "μ1 = μ2 = ... = μ14 = μ27"

(Nous rejetons H0 si la valeur p est inférieure à 5 % : Rejeter H0 signifie ici rejeter l'hypothèse selon laquelle la variable qualitative (prdtypecode) n'influence pas la variable quantitative (Taille_images_bits))

In [9]:
# TEST ANOVA
import statsmodels.api

df_with_img = df_with_img.rename(columns={'Taille image (bits)': 'Taille_image_bits'})
df_with_img

anova = statsmodels.formula.api.ols('Taille_image_bits ~ prdtypecode', data = df_with_img).fit()
table = statsmodels.api.stats.anova_lm(anova)
table

#Le p-value (PR (> F)) est inférieur à 5% --> taille de l'image affecte prdtypecode.
#Il y a bien la taille des images qui bougent en fonction de la catégorie.
#La taille de l'image est donc une variable à retenir pour une classification de data.

,df,sum_sq,mean_sq,F,PR(>F)
prdtypecode,1.0,1.290581e+11,1.290581e+11,678.58319,5.253152e-149
Residual,84914.0,1.614959e+13,1.901876e+08,NaN,NaN


##### Le test ANOVA nous suggère qu'il y a une différence significative dans la taille des images entre les différentes catégories de prdtypecode. 

#### Interprétation des résultats obtenus :

- df (degrés de liberté) :
    - Pour prdtypecode: 1 (nombre de catégories - 1) 
    - Pour Residual (résidus) : 84914 (nombres de lignes)
- sum_sq (somme des carrés) : 
    - Pour prdtypecode: 1.290581e+11 (somme des carrés entre les groupes) 
    - Pour Residual : 1.614959e+13 (somme des carrés des résidus)
- mean_sq (moyenne des carrés) : 
    - Pour prdtypecode: 1.290581e+11 (somme des carrés entre les groupes divisée par les degrés de liberté)
    - Pour Residual : 1.901876e+08 (somme des carrés des résidus divisée par les degrés de liberté résiduels)
- F (statistique de test) : 678.58 (ratio de variance entre les groupes sur la variance à l'intérieur des groupes)
- PR(>F) (valeur p) : 5.25e-149 (probabilité que la statistique de test soit au moins aussi extrême que celle observée, sous l'hypothèse nulle)

#### Interprétation :

La valeur p (PR(>F)) est très proche de zéro, bien en dessous du seuil de 0.05, ce qui suggère que la variable prdtypecode a une influence significative sur la taille des images. L'hypothèse nulle (H0) selon laquelle les moyennes des tailles d'images sont égales entre les catégories de prdtypecode est rejetée. En conséquence, la taille de l'image semble varier significativement en fonction de la catégorie prdtypecode. Cela renforce l'idée que la taille des images est une variable pertinente pour la classification des données

### **3. Préparation pour classification**

In [10]:
df = df_with_img[['Taille_image_bits', 'prdtypecode']]

df.head()

,Taille_image_bits,prdtypecode
0,14010,10
1,14854,2280
2,6898,50
3,14404,1280
4,20435,2705
